# Statement

Butano líquido es bombeado a un vaporizador como líquido saturado a una presión de 1.88 MPa. El butano sale del intercambiador como vapor húmedo con 90% de calidad y esencialmente a la misma presión de alimentación. Determine la carga calórica en el vaporizador por gramo de butano alimentado.

Datos del butano:
- Temperatura normal de ebullición = 272.2 K
- Punto triple: T = 134.6 K, P = 0.7 Pa
- Punto crítico: T = 425.1 K, P = 37.96 bar

In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

/Users/aliglara/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Solución

## Determinacion de los parámetros de la ecuación de Antoine
Se requiere una expresión para calcular la temperatura a la cual se encuentra el butano alimentado. Como el estado es de saturación, se emplea la ecuación de Antoine para determinar dicha temperatura.

Ahora bien, se conoce que el logaritmo natural de la presión de saturación de un componente puro varía con el inverso de la temperatura absoluta.

$$
\ln P^\text{sat} = A - \frac{B}{T+C}
$$

Como se desconoce los parámetros de la ecuación de Antoine, se emplea los datos de saturación datos para determinarlos. Existen diversas formas de resolver esto, en este ejercicio se utiliza algebra lineal para resolver un sistema de ecuaciones no lineal.

El primer paso, seria linealizar el sistema de ecuaciones

$$
\begin{aligned}
\ln P \left(\frac{T + C}{T} \right) & = A \left(\frac{T + C}{T} \right) - \frac{B}{T+C}\left(\frac{T + C}{T} \right) \\
\ln P + \frac{C \ln P}{T} & = A + \frac{AC}{T} - \frac{B}{T} \\
\ln P \left( 1+ \frac{C}{T}\right) & = A + \left(AC - B\right)\frac{1}{T} \\
\ln P & = A + \left(AC - B - C \ln P\right)\frac{1}{T}
\end{aligned}
$$

De esta manera, se observa que es una linea recta cuya pendiente es igual a $ \left(AC - B - C \ln P\right)$ y con corte igual a $A$.

Esto se puede reescribir para resolver, por ejemplo, mediante algebra lineal $a x = b$. Entonces:

\begin{equation}
\ln P = A + \left(AC - B\right)\frac{1}{T} -  C \:\frac{\ln P}{T}
\end{equation}

De forma matricial, seria:

\begin{equation*}
\left[ \begin{array}{ccc}
1 & \cfrac{1}{T_1} & \cfrac{\ln P_1}{T_1} \\
1 & \cfrac{1}{T_2} & \cfrac{\ln P_2}{T_2} \\
1 & \cfrac{1}{T_3} & \cfrac{\ln P_3}{T_2} \end{array} \right] \: \left[ \begin{array}{c}
A  \\
AC - B \\
C \end{array} \right] = \left[ \begin{array}{c}
\ln P_1  \\
\ln P_2 \\
\ln P_3 \end{array} \right]
\end{equation*}

In [2]:
# Datos del enunciado
#                Temp,  Presion
data = np.array([[134.6, 0.7], 
                 [272.7, 1.01325e5], 
                 [425.1, 3796000]])

In [3]:
# Matriz a
a = np.ones((3,3))
a[:,1] = 1/data[:,0]
a[:,2] = -np.log(data[:,1])/data[:,0]
print(a)

[[ 1.          0.00742942  0.00264989]
 [ 1.          0.00366703 -0.04226655]
 [ 1.          0.00235239 -0.0356374 ]]


In [4]:
# Vector b
b = np.log(data[:,1])
print(b)

[-0.35667494 11.52608845 15.14945844]



Para la solución numérica de los parámetros de Antoine, se emplea la funcion *solve* modulo de **linalg** from Numpy

In [5]:
# Solucion de vector x

x = np.linalg.solve(a,b)

# Determinacion de parametros de Antoine
A = x[0]
C = x[2]
B = A*C - x[1]

print(("Las constantes de la ecuacion de Antoine son: \n" 
       "A = {:.2f}, B = {:.2f} y C = {:.2f}".format(A, B, C)))

Las constantes de la ecuacion de Antoine son: 
A = 21.07, B = 2376.59 y C = -23.68


## Determinacion de la carga calorica

Ahora para determinar el calor de vaporizacion se relaciona la ecuacion de Antoine con la ecuación de Clausius-Clapeyron

\begin{aligned}
\left(\frac{\partial P^\text{sat}}{\partial T}\right)_{\Delta G =0 } & = \frac{\Delta H^\text{vap}}{R\,T^2} \\
\frac{B}{\left( T + C \right)^2} & = \frac{\Delta H^\text{vap}}{R\,T^2} 
\end{aligned}

Lo que se reduce a un sistema de ecuaciones con dos incognitas: la temperatura de saturacion del sistema y el cambio de entalpia de vaporizacion de la sustancia.

$$
\begin{aligned}
& \ln P^\text{sat} = A - \frac{B}{T+C} \\
& \frac{B}{\left( T + C \right)^2} = \frac{\Delta H^\text{vap}}{R\,T^2} 
\end{aligned}
$$

In [6]:
def sistema_ecuaciones(x, *args):
    '''implementacion del sistema de ecuaciones
    '''
    temp, delta_h = x
    presion, a, b, c, r_gas = args
    
    f1 = np.log(presion) - (a - b/(temp + c))
    f2 = b/(temp + c)**2 - delta_h/r_gas/temp**2
    
    return [f1, f2]

In [7]:
R_gas = 8.31416
Presion_feed = 1.88e6
calidad = .9
Peso_Molecular = 58

parametros = (Presion_feed, A, B, C, R_gas)
Temperature_feed, Delta_vapor = fsolve(sistema_ecuaciones, [350, 2000], args=parametros)

In [8]:
carga_calorica = Delta_vapor * calidad / Peso_Molecular
print("La carga calorica en el vaporizador es {:.2f} J/g".format(Delta_vapor * calidad / Peso_Molecular))

La carga calorica en el vaporizador es 348.42 J/g
